# TP1 - Parte 3 - Comunicación y Concurrencia

# **Python**
Este ejercicio comunica tres procesos emparentados A, B y C (Padre e Hijos) a través de tuberías (Pipe)

El programa A (Padre) lee desde un archivo los registros de ventas de sus dos sucursales y envia
la información leída a los programas B y C (Hijos) a través de las tuberías. Los registros de la sucursal 1 son enviados por el PIPE 1 hacia B y los de la sucursal 2 por el PIPE 2 hacia C.

El archivo sales.txt contiene los registros de ventas y el formato de cada registro es:   
SUCURSAL (1-2), FECHA (DD-MM-AAAA), PRODUCTO (Nombre), MONTO.

Cada proceso hijo procesa la información recibida y emite la siguiente salida:  
• Número de sucursal  
• Monto total  
• Producto más vendido  
• Fecha con mayor cantidad de ventas y la cantidad (Máximo tres en caso de empate).    
• Fecha con mayor monto vendido y el monto (Máximo tres en caso de empate).  

In [ ]:
%%writefile sales.txt
SUCURSAL1;12-03-2023;Pan;255.78
SUCURSAL2;05-07-2022;Tarta;504.12
SUCURSAL1;30-11-2021;Croissant;307.64
SUCURSAL2;20-06-2024;Pastel;621.34
SUCURSAL1;28-09-2022;Bollo;193.55
SUCURSAL2;14-01-2023;Baguette;415.82
SUCURSAL1;05-11-2024;Tarta;815.18
SUCURSAL2;10-03-2023;Galleta;720.66
SUCURSAL1;03-08-2021;Empanada;60.29
SUCURSAL2;09-05-2024;Bollo;808.95
SUCURSAL1;15-04-2023;Baguette;891.11
SUCURSAL2;18-10-2022;Pan;178.44
SUCURSAL1;25-07-2022;Pastel;819.62
SUCURSAL2;22-06-2023;Croissant;879.03
SUCURSAL1;11-01-2022;Pan;996.19
SUCURSAL2;07-09-2024;Galleta;439.54
SUCURSAL1;29-05-2021;Tarta;603.75
SUCURSAL2;01-12-2021;Empanada;75.04
SUCURSAL1;16-02-2023;Croissant;401.81
SUCURSAL2;04-03-2024;Bollo;57.88
SUCURSAL1;19-09-2023;Pastel;168.33
SUCURSAL2;23-10-2023;Galleta;575.43
SUCURSAL1;06-08-2022;Empanada;245.21
SUCURSAL2;17-12-2023;Tarta;433.08
SUCURSAL1;08-05-2024;Pan;796.49
SUCURSAL2;19-04-2021;Baguette;457.21
SUCURSAL1;27-07-2023;Bollo;24221.96
SUCURSAL2;02-08-2021;Pan;349.11
SUCURSAL1;10-09-2022;Galleta;258.16
SUCURSAL2;28-02-2024;Baguette;343.69
SUCURSAL1;13-10-2022;Pastel;828.31
SUCURSAL2;26-12-2021;Tarta;715.05
SUCURSAL1;24-05-2024;Pan;537.81
SUCURSAL2;21-11-2023;Bollo;294.57
SUCURSAL1;09-02-2022;Baguette;673.97
SUCURSAL2;03-09-2024;Galleta;67.88
SUCURSAL1;22-06-2023;Tarta;816.47
SUCURSAL2;30-05-2022;Pastel;511.13
SUCURSAL1;28-12-2023;Pan;803.31
SUCURSAL2;12-10-2023;Empanada;464.89
SUCURSAL1;17-08-2024;Baguette;59.44
SUCURSAL2;20-01-2022;Galleta;262.26
SUCURSAL1;05-07-2021;Bollo;355.35
SUCURSAL2;23-02-2023;Pan;581.15
SUCURSAL1;31-03-2022;Pastel;854.26
SUCURSAL2;14-09-2021;Tarta;697.03

Writing sales.txt


In [ ]:
%%writefile pipes.py
import os
from multiprocessing import Pipe

FILE_PATH = 'sales.txt'
CHILD = 0

def printSales(salesStats):
    print(f"Sucursal {salesStats['office']}:")
    print(f"Número de sucursal: {salesStats['office']}")
    print(f"Monto total: {salesStats['amount']}")
    print(f"Producto más vendido: {salesStats['bestSellingProductPriceName']}")
    print(f"Fecha con mayor cantidad de ventas: {salesStats['mostSelledDate']} - Cantidad: {salesStats['mostSelledDateCount']}")
    print(f"Fecha con mayor monto vendido: {salesStats['highestProfitDate']} - Monto: {salesStats['highestProfitDateSalesMount']}")

def getSalesStats(recvAB):
    amount = 0
    bestSellingProductPrice = {"price": 0, "name": None}
    dateSales = {}

    msg = recvAB.recv()
    while(msg is not None ):
        office, date, name, price = msg
        price = float(price)

        amount += price
        if(price > bestSellingProductPrice['price']): bestSellingProductPrice =  {"price": price, "name": name}
        dateSales[date] = dateSales[date] + [price] if isinstance(dateSales.get(date) , list) else [price]
        msg = recvAB.recv()

    mostSelledDate = max(dateSales, key=lambda date: len(dateSales[date]))
    highestProfitDate = max(dateSales, key=lambda date: sum(dateSales[date]))
    mostSelledDateCount = len(dateSales[mostSelledDate])
    bestSellingProductPriceName = bestSellingProductPrice['name']
    highestProfitDateSalesMount = sum(dateSales[highestProfitDate])

    return {
        "office": office,
        "amount": amount,
        "bestSellingProductPriceName": bestSellingProductPriceName,
        "mostSelledDate": mostSelledDate,
        "mostSelledDateCount": mostSelledDateCount,
        "highestProfitDate": highestProfitDate,
        "highestProfitDateSalesMount": highestProfitDateSalesMount
    }

def main():
    recvAB, sendAB = Pipe(False)
    recvAC, sendAC = Pipe(False)
    recvBC, sendBC = Pipe(False)

    processB = os.fork()
    if processB == CHILD:
        sendAB.close()
        sendBC.close()
        salesStats = getSalesStats(recvAB)
        recvBC.recv()
        printSales(salesStats)
        recvBC.close()
        recvAB.close()
        os._exit(0)

    processC = os.fork()
    if processC == CHILD:
        sendAC.close()
        recvBC.close()
        salesStats = getSalesStats(recvAC)
        printSales(salesStats)
        sendBC.send(None)
        recvAC.close()
        sendBC.close()
        os._exit(0)

    recvAB.close()
    recvAC.close()
    with open(FILE_PATH, 'r') as file:
        for line in file:
            line = line.strip().split(';')
            office, date, name, price = line
            if office == 'SUCURSAL2':
                sendAB.send(line)
            if office == 'SUCURSAL1':
                sendAC.send(line)
    sendAB.send(None)
    sendAC.send(None)
    sendAB.close()
    sendAC.close()

    os.wait()
    os.wait()
    os._exit(0)

if __name__ == '__main__':
    main()

Writing pipes.py


In [ ]:
!python pipes.py

Sucursal SUCURSAL1:
Número de sucursal: SUCURSAL1
Monto total: 35963.990000000005
Producto más vendido: Bollo
Fecha con mayor cantidad de ventas: 12-03-2023 - Cantidad: 1
Fecha con mayor monto vendido: 27-07-2023 - Monto: 24221.96
Sucursal SUCURSAL2:
Número de sucursal: SUCURSAL2
Monto total: 10453.3
Producto más vendido: Croissant
Fecha con mayor cantidad de ventas: 05-07-2022 - Cantidad: 1
Fecha con mayor monto vendido: 22-06-2023 - Monto: 879.03


# **Conclusión**



 No nos topamos con ningun inconveniente en este ejercicio, las pipes son bastantes sencillas de implementar en Python con la biblioteca **multiprocessing.Pipe**, la cual nos provee de las funciones *send()*, *recv()* y *close()* que permiten un correcto funcionamiento.